In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_1 = pd.read_csv(r"..\..\data\clean\all_clients.csv")
df_2 = pd.read_csv(r"..\..\data\clean\control_filter.csv")
df_3 = pd.read_csv(r"..\..\data\clean\test_filter.csv")

In [ ]:
def valid_session(df: pd.DataFrame) -> pd.DataFrame:
    # Convert the 'date_time' column to datetime format
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    # Get the most recent session for each client (the latest visit_id per client)
    most_recent_sessions = df.loc[df.groupby('client_id')['date_time'].idxmax()]
    
    # Now filter the data to include only the most recent sessions
    df_recent = df[df['visit_id'].isin(most_recent_sessions['visit_id'])]
    
    # Sort by client_id, visitor_id, and date_time to make sure steps are in correct order
    df_recent = df_recent.sort_values(by=['client_id', 'visitor_id', 'date_time'])

    return df_recent


df_1 = valid_session(df_1)


step_mapping = {
        'start': 0,
        'step_1': 1,
        'step_2': 2,
        'step_3': 3,
        'confirm': 4
    }
def step_map(df):
    df["process_step_num"] = df["process_step"].map(step_mapping)
    df = df.sort_values(by=["client_id", "process_step_num"], ascending = True)

    return df

step_map(df_1)

df_1.tail()

In [ ]:
def total_time_again(df):

    # Assuming you have the DataFrame 'df'
    # Step 1: Convert 'date_time' to datetime
    df['date_time'] = pd.to_datetime(df['date_time'])

    # Step 2: Sort the DataFrame by visit_id and date_time to ensure the sequence is correct
    df.sort_values(by=['visit_id', 'date_time'], inplace=True)

    # Step 3: Calculate the time difference between consecutive rows
    df['time_diff'] = df.groupby('visit_id')['date_time'].diff()

    # Step 4: Filter out rows where time_diff is NaT (first row of each visit_id)
    df = df.dropna(subset=['time_diff'])

    # Step 5: Calculate average time spent per process_step
    average_time_per_step = df.groupby('process_step')['time_diff'].mean()

    # Step 6: Convert time difference to minutes or seconds
    average_time_per_step = average_time_per_step.dt.total_seconds() #/ 60  # In minutes
    average_time_per_step = average_time_per_step.round(2)
    print(average_time_per_step)


total_time_again(df_1)